In [1]:
from fisher_py.data.business import Scan
from fisher_py import RawFile
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
raw = RawFile("c:/20240708_chm134_Nadig_MAbPac-04A_Proteo06_Group01_Time00_biorep01_techrep01.raw")

In [3]:
def cast_func(scan_number):
    scan_masses = []
    scan_intensities = []
    data_masses = []
    data_intensities = [] 


    raw_scan = Scan.from_file(raw._raw_file_access, scan_number=scan_number).centroid_scan
    scan_masses = raw_scan.masses
    scan_intensities = raw_scan.intensities

    for i in range(4000, 20000): 
        data_intensities.append(0) 
        data_masses.append(i/10)

    for j in range(0,len(scan_masses)):
        for i in range(0,len(data_masses)):
            if (i+ 4000 + 1)/10 > scan_masses[j] > (i + 4000)/10: 
                data_intensities[i] = data_intensities[i] + scan_intensities[j]
    return(list(data_intensities))

In [43]:
X = [cast_func(1009), cast_func(1010)]
round(cosine_similarity(X)[1][0], 2)

0.0